# Pydantic を使ったDictのバリデーション

今まで、補完を効かせるためにTypedDictを使ってきたが、Pydanticに食わせればよい

In [34]:
import dataclasses
from typing import cast

from datetime import datetime
import pydantic

In [35]:
class User(pydantic.BaseModel):
    id: int
    name: str
    signup_ts: datetime

@dataclasses.dataclass
class NUser:
    id: int
    name: str
    signup_ts: datetime


In [36]:
u1 = User(id=1, name='John Doe', signup_ts=datetime.fromisoformat('2017-06-01T12:22:00'))
u2 = NUser(id=1, name='John Doe', signup_ts=datetime.fromisoformat('2017-06-01T12:22:00'))

In [37]:
u1.test = ""

ValueError: "User" object has no field "test"

In [38]:
u2.test = ""

Pydanticのモデルを使うと、パラメータ増やしたりできなくなる

In [39]:
un = NUser(id=1, name='John Doe', signup_ts=100)
u = User(id=1, name='John Doe', signup_ts=100)

実行はできて、動的には検証してくれてない？？？

In [31]:
v = {
    "id": 100,
    "name": 200,
    "signup_ts": datetime.fromisoformat("2024-04-30T12:22:00"),
}
un = NUser(**v)
print(un)
u = User(**v)
print(u)

NUser(id=100, name=200, signup_ts=datetime.datetime(2024, 4, 30, 12, 22))


ValidationError: 1 validation error for User
name
  Input should be a valid string [type=string_type, input_value=200, input_type=int]
    For further information visit https://errors.pydantic.dev/2.7/v/string_type

エラーになってくれる

In [32]:
v = {
    "id": 100,
    "name": "200",
    "signup_ts": "2024-04-30T12:22:00",
}
u = User(**v)
print(u)

id=100 name='200' signup_ts=datetime.datetime(2024, 4, 30, 12, 22)


変換できるなら、良い感じに変換しておいてくれる

In [40]:
@pydantic.dataclasses.dataclass
class DUser:
    id: int
    name: str
    signup_ts: datetime

In [42]:
v = {
    "id": 100,
    "name": "200",
    "signup_ts": "2024-04-30T12:22:00",
}
u = DUser(**v)
print(u)

DUser(id=100, name='200', signup_ts=datetime.datetime(2024, 4, 30, 12, 22))


## 構造的なやつ

In [43]:

@pydantic.dataclasses.dataclass
class Pos:
    lat: pydantic.types.StrictFloat
    lon: pydantic.types.StrictFloat

@pydantic.dataclasses.dataclass
class Response:
    route: list[Pos]

In [45]:
p1=Pos(lat=1.0, lon=2.0)
p2=Pos(lat=3.0, lon=4.0)

Response(route=[p1, p2])

Response(route=[Pos(lat=1.0, lon=2.0), Pos(lat=3.0, lon=4.0)])

In [46]:
Response(route=[[(1,2),(3,4)]])

ValidationError: 1 validation error for Response
route.0
  Input should be a dictionary or an instance of Pos [type=dataclass_type, input_value=[(1, 2), (3, 4)], input_type=list]
    For further information visit https://errors.pydantic.dev/2.7/v/dataclass_type

In [51]:
@pydantic.dataclasses.dataclass
class Response2:
    route: list[tuple[float, float]]

@dataclasses.dataclass
class NResponse2:
    route: list[tuple[float, float]]

In [48]:
Response2(route=[(1.1,2.2),(3.3,4.4)])

Response2(route=[(1.1, 2.2), (3.3, 4.4)])

In [49]:
Response2(route=[(1.1,2.2,3.3),(3.3,4.4)])

ValidationError: 1 validation error for Response2
route.0
  Tuple should have at most 2 items after validation, not 3 [type=too_long, input_value=(1.1, 2.2, 3.3), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.7/v/too_long

In [52]:
NResponse2(route=[(1.1,2.2,3.3),(3.3,4.4)])

NResponse2(route=[(1.1, 2.2, 3.3), (3.3, 4.4)])

従来のdataclassなら通ってしまう